<a href="https://colab.research.google.com/github/Prathamesh-Ghatole/Monthly-Budget-Analysis/blob/main/Monthly_Budget_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use Monthly Budget Data, and figure out insights.

Q1. Describe the Total amount of money spent

Q2. Which day accounts for the most entries?

Q3. Which category accounts for the most entries?

Q4. Which day accounts for the most amount of money spent?

Q5. Which category accounts for the most amount of money spent?

Q6. What's the amount of money spent each day?

Q7. On which day & date was maximum amount of money spent?

Q8. What's the average amount of daily spendings?

Q9. What are some top key-words for expenses?

In [133]:
import pandas as pd
import plotly.express as px

In [134]:
df = pd.read_csv('/content/Budget_Dec_2021.csv')

#Preview df
# df

#Dropping useless columns
df = df.drop('Comment', axis=1)

# preview df again
df.head(10)

,Expense,Amount,Category,Date,Payment Mode
0,Patties for breakfast,20.0,Food,"December 3, 2021",Online
1,Kurkure x2,20.0,Food,"December 3, 2021",Cash
2,Vending machine chips,10.0,Food,"December 4, 2021",Cash
3,Paid for washing clothes,30.0,Laundry and Maintainance,"December 4, 2021",Cash
4,50rs snacks from vending machine 😑,45.0,Food,"December 4, 2021",Cash
5,Hospital,498.0,Emergency,"December 6, 2021",Cash
6,Snacc from vending machine,20.0,Food,"December 7, 2021",Cash
7,Chaddar washing,20.0,Laundry and Maintainance,"December 8, 2021",Cash
8,Kurkure + Lassi,30.0,Food,"December 8, 2021",Cash
9,"""Dinner""",50.0,Food,"December 9, 2021",Cash


In [135]:
#Check rows with atleast 1 empty value
df[df.isna().any(axis=1)]

,Expense,Amount,Category,Date,Payment Mode
14,Dinner,190.0,Food,"December 12, 2021",NaN
29,100DaysOfCode Party 🥲,90.0,"Food, Personal","December 21, 2021",NaN
30,Coffee + Snax,60.0,Food,"December 22, 2021",NaN
33,Dinner,60.0,Food,NaN,Cash
40,Midnight snacks,10.0,Food,NaN,Cash
47,NaN,NaN,NaN,NaN,NaN
48,NaN,NaN,NaN,NaN,NaN


In [136]:
#Dropping useless rows
df = df.drop(index = [47, 48], axis = 0)

#Filling nan values column-wise
df['Payment Mode'].fillna(method = 'ffill', inplace=True)
df['Date'].fillna(method = 'bfill', inplace=True)
df['Date'].fillna(method = 'bfill', inplace=True)

#AGAIN, Check rows with atleast 1 empty value
df[df.isna().any(axis=1)]
#No NaN values left.

,Expense,Amount,Category,Date,Payment Mode


In [137]:
#Changing the Date column to date-time format
df.Date = pd.DatetimeIndex(df.Date)
df.head()

,Expense,Amount,Category,Date,Payment Mode
0,Patties for breakfast,20.0,Food,2021-12-03,Online
1,Kurkure x2,20.0,Food,2021-12-03,Cash
2,Vending machine chips,10.0,Food,2021-12-04,Cash
3,Paid for washing clothes,30.0,Laundry and Maintainance,2021-12-04,Cash
4,50rs snacks from vending machine 😑,45.0,Food,2021-12-04,Cash


In [138]:
#Splitting Date column into day_of_week and weekdays.
df['Date_day'] = df.Date.dt.day_of_week
df['Weekday'] = df.Date.dt.day_name()
df.head()

,Expense,Amount,Category,Date,Payment Mode,Date_day,Weekday
0,Patties for breakfast,20.0,Food,2021-12-03,Online,4,Friday
1,Kurkure x2,20.0,Food,2021-12-03,Cash,4,Friday
2,Vending machine chips,10.0,Food,2021-12-04,Cash,5,Saturday
3,Paid for washing clothes,30.0,Laundry and Maintainance,2021-12-04,Cash,5,Saturday
4,50rs snacks from vending machine 😑,45.0,Food,2021-12-04,Cash,5,Saturday


In [139]:
#Splitting Category on commas, and keeping a list of items per column
splt = lambda x: x.split(', ')
df.Category = df.Category.map(splt)
df.head()

,Expense,Amount,Category,Date,Payment Mode,Date_day,Weekday
0,Patties for breakfast,20.0,[Food],2021-12-03,Online,4,Friday
1,Kurkure x2,20.0,[Food],2021-12-03,Cash,4,Friday
2,Vending machine chips,10.0,[Food],2021-12-04,Cash,5,Saturday
3,Paid for washing clothes,30.0,[Laundry and Maintainance],2021-12-04,Cash,5,Saturday
4,50rs snacks from vending machine 😑,45.0,[Food],2021-12-04,Cash,5,Saturday


# Q1. Describe the total amount of money spent.

In [140]:
# print(df.Amount.describe())

print('Total Entries:', len(df.Amount), '\n')
print('Total Monthly Spendings:', df.Amount.sum(), '\n')
print('Max entry amount:', df.Amount.max(), '\n')
print('Min entry amount:', df.Amount.min(), '\n')
print('Mean entry amount:', df.Amount.mean(), '\n')
print('Median entry amount:', df.Amount.median(), '\n')
print('Mode entry amount:', df.Amount.mode(), '\n')

Total Entries: 47 

Total Monthly Spendings: 3119.0 

Max entry amount: 498.0 

Min entry amount: 10.0 

Mean entry amount: 66.36170212765957 

Median entry amount: 40.0 

Mode entry amount: 0    20.0
dtype: float64 



Q2. Which day accounts for the most entries?

Q3. Which category accounts for the most entries?

In [141]:
# Simple function to traverse a column, and update values in given dictionary
def traverse_update(ls, d):
  for item in ls:
    if item not in d.keys():
      d[item] = 0
    else:
      d[item] += 1

#Dict declaration
day_counts, cat_counts = {}, {}
#mapping traverse_update on Categories column to get values.
#using lambda function to pass 2 values to map.
df.Category.map(lambda x: traverse_update(ls = x,d=cat_counts))
#Note: here x is wrapped in [] since traverse_update func only accepts lists as input.
df.Weekday.map(lambda x: traverse_update(ls = [x],d=day_counts))

#Counts for each entry
print(cat_counts)
print()
print(day_counts)

{'Food': 39, 'Laundry and Maintainance': 1, 'Emergency': 0, 'Travel': 0, 'Entertainment': 0, 'Personal': 8, 'Misc': 0}

{'Friday': 8, 'Saturday': 5, 'Monday': 4, 'Tuesday': 6, 'Wednesday': 6, 'Thursday': 4, 'Sunday': 7}


In [142]:
# px.bar(x=list(cat_counts.keys()), y=list(cat_counts.values()))
n_entry_day = px.histogram(x=list(day_counts.keys()), y=list(day_counts.values()), title = 'Number of Entries for each Week Day')
n_entry_cat = px.histogram(x=list(cat_counts.keys()), y=list(cat_counts.values()), title = 'Number of Entries for each Category')
# px.histogram()
n_entry_day.show()
n_entry_cat.show()

## Q4. Which day accounts for the most amount of money spent?

## Q5. Which category accounts for the most amount of money spent?

In [143]:
total_days, total_category = {}, {}
def append(key, value, d):
    if key not in d.keys():
      d[key] = 0
    else:
      d[key] += value

def get_total(row):
  amt = row['Amount']
  cat = row['Category']
  day = row['Weekday']

  append(day, amt, total_days)
  for category in cat:
    append(category, amt, total_category)
  
df.apply(get_total, axis=1)
print(total_days)
print()
print(total_category)

{'Friday': 230.0, 'Saturday': 210.0, 'Monday': 305.0, 'Tuesday': 400.0, 'Wednesday': 225.0, 'Thursday': 176.0, 'Sunday': 785.0}

{'Food': 2010.0, 'Laundry and Maintainance': 20.0, 'Emergency': 0, 'Travel': 0, 'Entertainment': 0, 'Personal': 571.0, 'Misc': 0}


In [144]:
total_weekday = px.pie(names=list(total_days.keys()), values=list(total_days.values()), title = 'Total amount for each Week Day')
total_category = px.pie(names=list(total_category.keys()), values=list(total_category.values()), title = 'Total amount for each Category')

total_weekday.show()
total_category.show()

In [ ]:
# 